# TensorFlow Assignment: Multilayer Perceptron (MLP) and Convolutional Neural Network (CNN)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: Rohith Kuditipudi

Now that you've run through a simple logistic regression model on MNIST, let's see if we can do better (Hint: we can). For this assignment, you'll build a multilayer perceptron (MLP) and a convolutional neural network (CNN), two popular types of neural networks, and compare their performance. Some potentially useful code:

In [41]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Import data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [42]:
# Helper functions for creating weight variables
def weight_variable(shape):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Tensorflow Functions that might also be of interest:
# tf.nn.sigmoid()
# tf.nn.relu()

### Multilayer Perceptron

Build a multilayer perceptron for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> fully connected (500 hidden units) -> nonlinearity (Sigmoid/ReLU) -> fully connected (10 hidden units) -> softmax

Skeleton framework for you to fill in (Code you need to provide is marked by `###`):

In [43]:
# Model Inputs
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# Define the graph
W_i = weight_variable([784,500])
b_i = bias_variable([500])

W_h = weight_variable([500,10])
b_h = bias_variable([10])

### Create your MLP here##
### Make sure to name your MLP output as y_mlp ###
h_1 = tf.nn.relu(tf.matmul(x,W_i)+b_i)
y_mlp = tf.nn.softmax(tf.matmul(h_1,W_h)+b_h)


# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_mlp))

# Optimizer
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_mlp, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Training regimen
    for i in range(4000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(100)
                validation_accuracy += (1/10) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))
        
        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

step 0, validation accuracy 0.146
step 250, validation accuracy 0.746
step 500, validation accuracy 0.743
step 750, validation accuracy 0.827
step 1000, validation accuracy 0.865
step 1250, validation accuracy 0.86
step 1500, validation accuracy 0.841
step 1750, validation accuracy 0.854
step 2000, validation accuracy 0.865
step 2250, validation accuracy 0.868
step 2500, validation accuracy 0.869
step 2750, validation accuracy 0.883
step 3000, validation accuracy 0.937
step 3250, validation accuracy 0.966
step 3500, validation accuracy 0.965
step 3750, validation accuracy 0.955
test accuracy 0.9619


#### Comparison

How do the sigmoid and rectified linear unit (ReLU) compare?

***

ReLU is better. It not only converges faster but also achieves better results. A possible explanation is that sigmoid is more prone to saturation (at least for large positive inputs), which leads to weak gradients.

***

### Convolutional Neural Network

Build a simple 2-layer CNN for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> CNN (32 5x5 filters) -> nonlinearity (ReLU) ->  (2x2 max pool) -> CNN (64 5x5 filters) -> nonlinearity (ReLU) -> (2x2 max pool) -> fully connected (1024 hidden units) -> nonlinearity (ReLU) -> fully connected (10 hidden units) -> softmax

Some additional functions that you might find helpful:

In [44]:
# Convolutional neural network functions
def conv2d(x, W):
    """conv2d returns a 2d convolution layer with full stride."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def fc(x, W):
    """fc returns a fully connected layer; the shape of W should be [inputdim1,inputdim2,input_depth,output_hidden_units]"""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

def max_pool_2x2(x):
    """max_pool_2x2 downsamples a feature map by 2X."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Tensorflow Function that might also be of interest:
# tf.reshape()

Skeleton framework for you to fill in (Code you need to provide is marked by `###`):

*Hint: Convolutional Neural Networks are spatial models. You'll want to transform the flattened MNIST vectors into images for the CNN. Similarly, you might want to flatten it again sometime before you do a softmax. You also might want to look into the  [conv2d() documentation](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d) to see what shape kernel/filter it's expecting.*

In [ ]:
# Model Inputs
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

print(x.shape)
# Define the graph
W_i = weight_variable([5, 5, 1, 32])
W_h1 = weight_variable([5,5,32,64])
W_h2 = weight_variable([7,7,64,1024])
W_h3 = weight_variable([1,1,1024,10])

#Image -> CNN (32 5x5 filters) -> nonlinearity (ReLU) -> 
#(2x2 max pool) -> CNN (64 5x5 filters) -> nonlinearity (ReLU) -> 
#(2x2 max pool) -> fully connected (1024 hidden units) -> nonlinearity (ReLU) -> 
#fully connected (10 hidden units) -> softmax

### Create your CNN here##
### Make sure to name your CNN output as y_conv ###
x_i = tf.reshape(x,[-1,28,28,1])
print(x_i.shape)
h1 = max_pool_2x2(tf.nn.relu(conv2d(x_i,W_i)))
print(h1.shape)
h2 = max_pool_2x2(tf.nn.relu(conv2d(h1,W_h1)))
print(h2.shape)
h3 = tf.nn.relu(fc(h2,W_h2))
print(h3.shape)
o = tf.nn.softmax(conv2d(h3,W_h3))
print(o.shape)
y_conv = tf.reshape(o,[-1,10])
print(y_conv.shape)

# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

# Optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Training regimen
    for i in range(10000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(50)
                validation_accuracy += (1/10) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))
        
        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

(?, 784)
(?, 28, 28, 1)
(?, 14, 14, 32)
(?, 7, 7, 64)
(?, 1, 1, 1024)
(?, 1, 1, 10)
(?, 10)
step 0, validation accuracy 0.092
step 250, validation accuracy 0.662
step 500, validation accuracy 0.762
step 750, validation accuracy 0.804
step 1000, validation accuracy 0.75


Some differences from the logistic regression model to note:

- The CNN model might take a while to train. Depending on your machine, you might expect this to take up to half an hour. If you see your validation performance start to plateau, you can kill the training.

- The logistic regression model we used previously was pretty basic, and as such, we were able to get away with using the GradientDescentOptimizer, which performs implements the gradient descent algorithm. For more difficult optimization spaces (such as the ones deep networks pose), we might want to use more sophisticated algorithms. Prof David Carlson has a lecture on this later.
    
- Because of the larger size of our network, notice that our minibatch size has shrunk.
    
- We've added a validation step every 250 minibatches. This let's us see how our model is doing during the training process, rather than sit around twiddling our thumbs and hoping for the best when training finishes. This becomes especially significant as training regimens start approaching days and weeks in length. Normally, we validate on the entire validation set, but for the sake of time we'll just stick to 10 validation minibatches (500 images) for this homework assignment.

#### Comparison

How do the MLP and CNN compare in accuracy? Training time? Why would you use one vs the other? Is there a problem you see with MLPs when applied to other image datasets?

***

CNN is more accurate, but takes a lot longer to train because its a deep model and so has more parameters. I'd use a CNN for image data, or in general when I'm working with a dataset for which the properties of a CNN would help me (i.e. translation invariance). I'd use a shallow MLP for simple tasks, or when the properties of a CNN don't help me (e.g. I don't care about translation invariance if I'm working in OpenAI gym and my input is a state vector, not raw pixels).

It's also worth keeping in mind that MNIST is a pretty simple dataset, and MLPs don't scale well to datasets like ImageNet. I would expect to achieve better results with a CNN on ImageNet than I would with an MLP. One reason why CNNs scale better than MLPs is that the CNN architecture is specifically designed to leverage prior domain knowledge about images/object detection (e.g. translation invariance). Because of tricks like parameter sharing employed by CNNs, an MLP would likely require a lot more parameters to achieve comparable performance with a CNN.
***